In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import os
from copy import copy
import missingno as ms
#from dask.diagnostics import ProgressBar
#pbar = ProgressBar()
#pbar.register()
pd.options.display.float_format = '{:.4f}'.format

In [13]:
# Funckija pretvorbe ticker podataka u minutne podatke
def ticker_to_time(name):
    # Loada dataset pomocu daska posto su neki preveliki da bi stali u memoriju
    df = dd.read_csv('/home/filip/Desktop/Ruđer/Data/'+name+".csv",header=None,dtype={0: int,1: float,2: float})
    print("Loaded")
    # Pretvaranje UNIX timestampa u Datetime format i uklanjanje sekunda kako bi se
    # moglo grupirati timestampove po minutama
    # Uzete su vrijednosti starije od 2014 godine radio pouzdanosti podataka pošto u tom periodu 
    # postoje podaci s više exchangeva 
    df['Timestamp'] = df[0].astype('M8[s]')
    df.set_index('Timestamp')
    df['Timestamp'] = df['Timestamp'].dt.floor("min")
    df = df.loc[df["Timestamp"]>"2014-01-01 01:00:00"]
    print("Timestamp")
    # Agregiranje podataka 
    df = df.groupby("Timestamp").agg({1: ['mean',"first","last", 'std','count','min',"max"],2:["sum"]})
    print("Aggregated")
    # Pretvorba dask dataframea u pandas dataframe
    df = df.compute()
    print("Computed")
    # Klasične vrijednosti koje se računaju za vrijednost nekog asseta
    df["High"] = df[1]["max"]
    df["Low"] = df[1]["min"]
    df["Trades"] = df[1]["count"]
    df["Std"] = df[1]["std"]
    df["Weighted_Price"] = df[1]["mean"]
    df["Open"] = df[1]["first"]
    df["Close"] = df[1]["last"]
    df["Volume"] = df[2]["sum"]

    
    df.drop(columns=[1,2],inplace=True)
    df['Timestamp'] = df.index
    return df

In [9]:
# Učitavanje imena svih fileova u datoteci
exchanges = os.listdir("/home/filip/Desktop/Ruđer/Data/")
existing = os.listdir("/home/filip/Desktop/Ruđer/MinuteData/")
exchanges = [x.split(".")[0] for x in exchanges]
exchanges.remove("coinbaseEUR")
existing = [x.split("_")[0] for x in existing]
print(exchanges)
print(existing)

['coincheckJPY', 'bitstampUSD', 'coinbaseUSD', 'btcnCNY']
['bitstampUSD', 'coinbaseUSD']


In [14]:
# Pretvorba podataka i spremanje pretvorenih podataka
for i in exchanges:
    name = i.split(".")[0]
    if not name in existing:
        df = ticker_to_time(name)
        df.to_csv(index=False, path_or_buf="/home/filip/Desktop/Ruđer/MinuteData/"+name+"_1-min_data.csv")

Loaded
Timestamp
Aggregated
Computed
Loaded
Timestamp
Aggregated


/home/filip/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


Computed


In [ ]:
dolars = 0
volume = 0
time = 0
ticks = 0
maximum = 0
minimum = 100000
openn = 0
close = 0
price_sum = 0
dataframe_list = []
df = pd.DataFrame(columns=["Timestamp","Dolars","Volume","Ticks","Time","Max","Min","Open","Close","Sum"])
def dolar_bar(x):
    global dolars,volume,time,ticks,maximum,minimum,openn,close,dataframe_list,price_sum
    if dolars == 0:
        time = x[0]
        openn = x[1]
    dolars += x[1]*x[2]
    price_sum += x[1]
    volume += x[2]
    ticks += 1
    if x[1]>maximum:
        maximum = x[1]
    if x[1]<minimum:
        minimum = x[1]
    if dolars > 250000:
        dictionary = {
            "Timestamp":x[0],
            "Dolars":dolars,
            "Volume":volume,
            "Ticks":ticks,
            "Time":x[0]-time,
            "Max":maximum,
            "Min":minimum,
            "Open":openn,
            "Close": x[1],
            "Sum":price_sum
        }
        dataframe_list.append(dictionary)
        print(dictionary)
        dolars = 0
        volume = 0
        time = 0
        ticks = 0
        price_sum = 0
        maximum = 0
        minimum = 100000
        openn = 0
        close = 0

def dolar_df(name):
    global dataframe_list
    df = pd.read_csv('/home/filip/Desktop/Ruđer/Data/'+name,header=None,dtype={0: int,1: float,2: float})
    df['Timestamp'] = df[0].astype('M8[s]')
    df = df.set_index('Timestamp')
    df.apply(dolar_bar,axis=1)
    df = pd.DataFrame(dataframe_list)
    df['Timestamp'] = df["Timestamp"].astype('M8[s]')
    df["Price"] = df["Sum"] / df["Ticks"]
    return df

In [ ]:
# Učitavanje imena svih fileova u datoteci
exchanges = os.listdir("/home/filip/Desktop/Ruđer/Data/")
# Pretvorba podataka i spremanje pretvorenih podataka
for name in exchanges:
    print(name)
    df = dolar_df(name)
    df.to_csv(index=False, path_or_buf="/home/filip/Desktop/Ruđer/DolarData/"+name+"_dolar_data.csv")
    df.describe()